In [547]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotnine 
from plotnine import * 
from xgboost.sklearn import XGBRegressor as xgb 
from lightgbm.sklearn import LGBMRegressor as lgb 
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_log_error
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
from sklearn.preprocessing import OrdinalEncoder
import catboost as cat

In [548]:
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth',None) 
plotnine.options.figure_size = (15, 9.27)
theme_set(theme_gray(base_size=20))
sns.set(font_scale=1.2) 

**Background**: With 79 explanatory variables describing (almost) every aspect of residential homes in Ames, Iowa, predict the final price of each home.

**Objective**: Predict the final price of each home.

**Type**: Regression

**Scale**: 1460 for train; 1459 for test.

**Evaluation**: RMSE

In [549]:
df = pd.read_csv('/Users/wangshuo/Library/Mobile Documents/com~apple~CloudDocs/data/house-prices-advanced-regression-techniques/train.csv').drop("Id",axis=1)

Test = pd.read_csv('/Users/wangshuo/Library/Mobile Documents/com~apple~CloudDocs/data/house-prices-advanced-regression-techniques/test.csv')

# Exploratory Data Analysis

In [223]:
# Num_var_des、Cat_var_des
Num_var_des = [['MSSubClass','Identifies the type of dwelling involved in the sale.(20-190)'],
                            ['LotFrontage','Linear feet of street connected to property.'],
                            ['LotArea','Lot size in square feet.'],
                            ['OverallQual','Rates the overall material and finish of the house.(1-10)'],
                            ['OverallCond','Rates the overall condition of the house.(1-10)'],
                            ['YearBuilt','Original construction date.'],
                            ['YearRemodAdd','Remodel date (same as construction date if no remodeling or additions).'],
                            ['MasVnrArea','Masonry veneer area in square feet.'],
                            ['BsmtFinSF1','Type 1 finished square feet.'],
                            ['BsmtFinSF2','Type 2 finished square feet.'],
                            ['BsmtUnfSF','Unfinished square feet of basement area.'],
                            ['TotalBsmtSF','Total square feet of basement area.'],
                            ['1stFlrSF','First Floor square feet.'],
                            ['2ndFlrSF','Second floor square feet.'],
                            ['LowQualFinSF','Low quality finished square feet (all floors).'],
                            ['GrLivArea','Above grade (ground) living area square feet.'],
                            ['BsmtFullBath','Basement full bathrooms.'],
                            ['BsmtHalfBath','Basement half bathrooms.'],
                            ['FullBath','Full bathrooms above grade.'],
                            ['HalfBath','Half baths above grade.'],
                            ['BedroomAbvGr',np.nan],
                            ['KitchenAbvGr',np.nan],
                            ['TotRmsAbvGrd','Total rooms above grade (does not include bathrooms).'],
                            ['Fireplaces','Number of fireplaces.'],
                            ['GarageYrBlt','Year garage was built.'],
                            ['GarageCars','Size of garage in car capacity.'],
                            ['GarageArea','Size of garage in square feet.'],
                            ['WoodDeckSF','Wood deck area in square feet.'],
                            ['OpenPorchSF','Open porch area in square feet.'],
                            ['EnclosedPorch','Enclosed porch area in square feet.'],
                            ['3SsnPorch','Three season porch area in square feet.'],
                            ['ScreenPorch','Screen porch area in square feet.'],
                            ['PoolArea','Pool area in square feet.'],
                            ['MiscVal','Value of miscellaneous feature.'],
                            ['MoSold','Month Sold (MM).'],
                            ['YrSold','Year Sold (YYYY).']]

Cat_var_des = [['MSZoning','Identifies the general zoning classification of the sale.'],
                          ['Street','Type of road access to property.'],
                          ['Alley','Type of alley access to property.'],
                          ['LotShape','General shape of property.'],
                          ['LandContour','Flatness of the property.'],
                          ['Utilities','Type of utilities available.'],
                          ['LotConfig','Lot configuration.'],
                          ['*LandSlope','Slope of property.'],
                          ['Neighborhood','Physical locations within Ames city limits.'],
                          ['Condition1','Proximity to various conditions.'],
                          ['Condition2','Proximity to various conditions (if more than one is present).'],
                          ['BldgType','Type of dwelling.'],
                          ['HouseStyle','Style of dwelling.'],
                          ['RoofStyle','Type of roof.'],
                          ['RoofMatl','Roof material.'],
                          ['Exterior1st','Exterior covering on house.'],
                          ['Exterior2nd','Exterior covering on house (if more than one material).'],
                          ['MasVnrType','Masonry veneer type.'],
                          ['*ExterQual','Evaluates the quality of the material on the exterior.'],
                          ['*ExterCond','Evaluates the present condition of the material on the exterior.'],
                          ['Foundation','Type of foundation.'],
                          ['*BsmtQual','Evaluates the height of the basement.'],
                          ['*BsmtCond','Evaluates the general condition of the basement.'],
                          ['*BsmtExposure','Refers to walkout or garden level walls.'],
                          ['*BsmtFinType1','Rating of basement finished area.'],
                          ['*BsmtFinType2','Rating of basement finished area (if multiple types).'],
                          ['Heating','Type of heating.'],
                          ['*HeatingQC','Heating quality and condition.'],
                          ['*CentralAir','Central air conditioning.'],
                          ['Electrical','Electrical system.'],
                          ['*KitchenQual','Kitchen quality.'],
                          ['Functional','Home functionality (Assume typical unless deductions are warranted).'],
                          ['*FireplaceQu','Fireplace quality.'],
                          ['GarageType','Garage location.'],
                          ['*GarageFinish','Interior finish of the garage.'],
                          ['*GarageQual','Garage quality.'],
                          ['*GarageCond','Garage condition.'],
                          ['*PavedDrive','Paved driveway.'],
                          ['*PoolQC','Pool quality.'],
                          ['*Fence','Fence quality'],
                          ['MiscFeature','Miscellaneous feature not covered in other categories.'],
                          ['SaleType','Type of sale.'],
                          ['SaleCondition','Condition of sale.']] 

In [250]:
# EDA
def EDA(df,Test,Num_var_des=None,Cat_var_des=None):
    # basic information
    basic_information = [len(df),df.shape[1],df.select_dtypes(include='number').columns.shape[0],
                    df.select_dtypes(include='object').columns.shape[0],df.duplicated().sum()]

    basic_information = pd.DataFrame(data=basic_information,index=['Records','Variables','Num_var','Cat_var',
                                              'Duplicates']).reset_index().rename(columns={'index':'',0:'Number'})
    # numerical variables
    num_var = df.describe().transpose()
    
    # categorical variables
    cat_var = []
    
    for col in df.select_dtypes(include='object').columns:
        cat_var.append((col,df[col].value_counts(dropna=False).to_dict()))
        
    cat_var = pd.DataFrame(cat_var,columns=['Feature','Categories'])     
    
    # numerical variables description
    num_var_des = pd.DataFrame(data=Num_var_des,columns=['Feature','Des'])
    
    # numerical variables description
    cat_var_des = pd.DataFrame(data=Cat_var_des,columns=['Feature','Des'])
    
    # variables with missing values
    miss = []
    for col in df.columns:
        miss.append((col,
                      df[col].nunique(),
                      df[col].isnull().sum(),
                      df[col].isnull().sum()*100/df.shape[0],
                      df[col].value_counts(normalize=True,dropna=False).values[0]*100,
                      df[col].dtype))
    miss = pd.DataFrame(miss,columns=['Feature',
                'Unique_values','Missing_values_num','Missing_values_per','Biggest_cat','Type']).sort_values('Missing_values_per',
                                                                                    ascending=False)
    
    miss.query('Missing_values_num>0',inplace=True)
    
    # number of outliers
    num_outliers = []
    
    for col in df.select_dtypes('number').columns:
        top = df[col].mean()+3*df[col].std()
        bot = df[col].mean()-3*df[col].std()
        mean1 = df[col].quantile(0.25)
        mean2 = df[col].quantile(0.75)
        top2 = mean2 + 1.5*(mean2-mean1)
        bot2 = mean1 - 1.5*(mean2-mean1)
        num_outliers.append((col,
                 df[(df[col]>top) | (df[col]<bot)].shape[0],
                 df[(df[col]>top2) | (df[col]<bot2)].shape[0]))
        
    num_outliers = pd.DataFrame(o,columns=['Feature',
                'OutlierNumStd','OutlierNumIQR'])
        
    # distribution of variables
    dis_var = []

    for col in Test.select_dtypes('object').columns:
        dis_var.append((col,
                    str(set(df[col].unique())-set(Test[col].unique())),
                    str(set(Test[col].unique())-set(df[col].unique()))))
        
    dis_var = pd.DataFrame(dis_var,columns=['Feature','train-test','test-train']) 
    
    dis_var = dis_var[(dis_var['train-test']!='set()') | (dis_var['test-train']!='set()')]

    if Num_var_des != None:
        return basic_information,num_var,cat_var,num_var_des,cat_var_des,miss,num_outliers,dis_var
    else:
        return basic_information,num_var,cat_var,miss,num_outliers,dis_var

basic_information,num_var,cat_var,num_var_des,cat_var_des,miss,num_outliers,dis_var = EDA(df,Test,Num_var_des,Cat_var_des)

In [209]:
# EDA_Charts
def EDA_Charts(df):
    # correlation
    correlation = plt.figure(figsize=(25,25),dpi=200)
    sns.heatmap(df.select_dtypes('number').corr(),annot=True,cmap="Blues")
    
    # num_dis
    k = 0 
    num_dis,ax = plt.subplots(7,6,figsize=(45,35),dpi=150)
    
    for a in range(7):
        for b in range(6):
            sns.histplot(df.select_dtypes(include='number').iloc[:,k],ax=ax[a,b],bins=8)
            k+=1
            if k > df.select_dtypes(include='number').shape[1]-1:
                break
    
    # cat_dis
    k = 0 
    cat_dis,ax = plt.subplots(7,7,figsize=(45,35),dpi=150)
    
    for a in range(7):
        for b in range(7):
            sns.countplot(y=df.select_dtypes(include='object').iloc[:,k],ax=ax[a,b])
            k+=1
            if k > df.select_dtypes(include='object').shape[1]-1:
                break
                
    # target
    target,ax = plt.subplots(2,1,figsize=(15,9),dpi=150)
    
    sns.histplot(x='SalePrice',data=df,bins=12,ax=ax[0])
    sns.boxplot(x='SalePrice',data=df,ax=ax[1])
    
    # time
    time, ax = plt.subplots(4,1,figsize=(15,18),dpi=150)
    sns.scatterplot(x='YearBuilt',y='SalePrice',data=df,ax=ax[0])
    sns.scatterplot(x='YrSold',y='SalePrice',data=df,ax=ax[1])
    sns.scatterplot(x='YearRemodAdd',y='SalePrice',data=df,ax=ax[2])
    sns.scatterplot(x='GarageYrBlt',y='SalePrice',data=df,ax=ax[3]) 

    return correlation,num_dis,cat_dis,target,time


correlation,num_dis,cat_dis,target,time = EDA_Charts(df)

# Feature Engineering

In [570]:
## data preprocessing
# modfify categorical features
def modify_cat(df):
    df['Utilities'] = df['Utilities'].replace('NoSeWa',df['Utilities'].value_counts().index[0])
    df['Condition2'] = df['Condition2'].replace(['RRAe', 'RRNn', 'RRAn'],df['Condition2'].value_counts().index[0])
    df['HouseStyle'] = df['HouseStyle'].replace('2.5Fin',df['HouseStyle'].value_counts().index[0])
    df['RoofMatl'] = df['RoofMatl'].replace(['Roll', 'Metal', 'ClyTile', 'Membran'],df['RoofMatl'].value_counts().index[0])
    df['Exterior1st'] = df['Exterior1st'].replace(['ImStucc', 'Stone'],df['Exterior1st'].value_counts().index[0])
    df['Exterior2nd'] = df['Exterior2nd'].replace('Other',df['Exterior2nd'].value_counts().index[0])
    df['Heating'] = df['Heating'].replace(['OthW', 'Floor'],df['Heating'].value_counts().index[0])
    df['Electrical'] = df['Electrical'].replace('Mix',df['Electrical'].value_counts().index[0])
    df['GarageQual'] = df['GarageQual'].replace('Ex',df['GarageQual'].value_counts().index[0])
    df['PoolQC'] = df['PoolQC'].replace('Fa',df['PoolQC'].value_counts().index[0])
    df['MiscFeature'] = df['MiscFeature'].replace('TenC',df['MiscFeature'].value_counts().index[0])
    
    return df

# missing value
def miss_value(df):
    # number
    for col in df.select_dtypes('number'):
        df[col] = df[col].fillna(df[col].median())
    
    # category
    # greater than 80%
    df = df.drop(['PoolQC','MiscFeature','Alley','Fence'],axis=1)
    
    # another type
    a = ['FireplaceQu','GarageCond','GarageType','GarageFinish','GarageQual','BsmtFinType2','BsmtExposure','BsmtQual','BsmtCond','BsmtFinType1','MasVnrType']
    df[a] = df[a].fillna(-1)
    
    # mode
    b = ['Electrical']
    for i in b:
        df[i] = df[i].fillna(df[i].value_counts().index[0])
    
    return df
    
## feature transformation
# number feature
# bin
def bin(df):
    l = ['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond',
       'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2',
       'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
       'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
       'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces',
       'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF',
       'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal',
       'MoSold', 'YrSold']
    for i in l:
        df[i] = pd.qcut(df[i].rank(method='first'),10,labels=False)
        
    return df 

# log
def ft_log(df):
    l = ['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond',
       'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2',
       'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
       'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
       'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces',
       'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF',
       'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal',
       'MoSold', 'YrSold']
    for i in l:
        df[i] = np.log(df[i])
        
    return df
    

# Categorical feature 
def cat_trans(df):
    # one hot 
    a = ['LandSlope','ExterQual','ExterCond','BsmtQual','BsmtCond'
 ,'BsmtExposure','BsmtFinType1','BsmtFinType2','HeatingQC','KitchenQual','FireplaceQu',
 'GarageFinish','GarageQual','GarageCond','PavedDrive','PoolQC','Fence']
    
    b = set(df.select_dtypes('object').columns) - set(a)
    
    df = pd.concat((df.drop(b,axis=1),pd.get_dummies(df[b])),axis=1)
    
    # ordial
    df['LandSlope'] = df['LandSlope'].replace({'Gtl':0,'Mod':1,'Sev':2})
    df['ExterQual'] = df['ExterQual'].replace({'Po':0,'Fa':1,'TA':2,'Gd':3,'Ex':4})
    df['ExterCond'] = df['ExterCond'].replace({'Po':0,'Fa':1,'TA':2,'Gd':3,'Ex':4})
    df['BsmtQual'] = df['BsmtQual'].replace({'Po':0,'Fa':1,'TA':2,'Gd':3,'Ex':4})
    df['BsmtCond'] = df['BsmtCond'].replace({'Po':0,'Fa':1,'TA':2,'Gd':3,'Ex':4})
    df['BsmtExposure'] = df['BsmtExposure'].replace({'None':0,'No':1,'Mn':2,'Av':3,'Gd':4})
    df['BsmtFinType1'] = df['BsmtFinType1'].replace({'None':0,'Unf':1,'LwQ':2,'Rec':3,'BLQ':4,
                                                    'ALQ':5,'GLQ':6})
    df['BsmtFinType2'] = df['BsmtFinType2'].replace({'None':0,'Unf':1,'LwQ':2,'Rec':3,'BLQ':4,
                                                    'ALQ':5,'GLQ':6})
    df['HeatingQC'] = df['HeatingQC'].replace({'Po':0,'Fa':1,'TA':2,'Gd':3,'Ex':4})
    df['KitchenQual'] = df['KitchenQual'].replace({'Po':0,'Fa':1,'TA':2,'Gd':3,'Ex':4})
    df['FireplaceQu'] = df['FireplaceQu'].replace({'Po':0,'Fa':1,'TA':2,'Gd':3,'Ex':4})
    df['GarageFinish'] = df['GarageFinish'].replace({'None':0,'Unf':1,'RFn':2,'Fin':3})
    df['GarageQual'] = df['GarageQual'].replace({'Po':0,'Fa':1,'TA':2,'Gd':3,'Ex':4})
    df['GarageCond'] = df['GarageCond'].replace({'Po':0,'Fa':1,'TA':2,'Gd':3,'Ex':4})
    df['PavedDrive'] = df['PavedDrive'].replace({'N':0,'P':1,'Y':2})
    
    return df 



In [571]:
# feature engineering for train
def fe_train(df):
    df = df.copy()
    ## data preprocessing
    # modfify categorical features
    df = modify_cat(df)
    
    # missing value
    df = miss_value(df)
    
    ## feature transformation
    # number feature
    # bin
#     df = bin(df)

   # log
    df = ft_log(df)
    
    # Categorical 
    df = cat_trans(df)
    
    return df

In [ ]:
# feature engineering for test
def fe_test(df):
    df = df.copy()
    # data preprocessing
    # modfify categorical features
    df = modify_cat(df)
    
    # feature transformation
    # Categorical features
    # one hot
    df = one_hot(df)
    
    return df 

In [551]:
# catboost
def evl_catb(df):
    folds = KFold(n_splits=10,shuffle=True)
    df = fe_train(df)
    train = df.drop('SalePrice', axis=1)
    target = df['SalePrice']

    train_score,test_score,RMSE_train,RMSE_test = [],[],[],[]
    best_model = 0
    best_RMSE = 100
    for train_idx,test_idx in folds.split(train,target):
        x_train,y_train = train.iloc[train_idx,:],target[train_idx]
        x_test,y_test = train.iloc[test_idx,:],target[test_idx]
        
        model = cat.CatBoostRegressor().fit(x_train,y_train,eval_set=[(x_test,y_test)],use_best_model=True)
        
        RMSE = np.sqrt(mean_squared_log_error(y_test,model.predict(x_test)))
        
        if RMSE < best_RMSE:
            best_RMSE = RMSE
            best_model = model
        
        train_score.append(model.score(x_train,y_train))
        test_score.append(model.score(x_test,y_test))
        RMSE_train.append(np.sqrt(mean_squared_log_error(y_train,model.predict(x_train))))
        RMSE_test.append(RMSE)
    
    d = pd.DataFrame()
    d['train_score'] = train_score
    d['test_score'] = test_score
    d['RMSE_train'] = RMSE_train
    d['RMSE_test'] = RMSE_test
    print(d)
    return best_model

In [526]:
# xgb
def evl_xgb(df):
    folds = KFold(n_splits=10,shuffle=True)
    df = fe_train(df)
    train = df.drop('SalePrice', axis=1)
    target = df['SalePrice']

    train_score,test_score,RMSE_train,RMSE_test = [],[],[],[]
    best_model = 0
    best_RMSE = 100
    for train_idx,test_idx in folds.split(train,target):
        x_train,y_train = train.iloc[train_idx,:],target[train_idx]
        x_test,y_test = train.iloc[test_idx,:],target[test_idx]
        
        model = xgb().fit(x_train,y_train)
        
        RMSE = np.sqrt(mean_squared_log_error(y_test,model.predict(x_test)))
        
        if RMSE < best_RMSE:
            best_RMSE = RMSE
            best_model = model
        
        train_score.append(model.score(x_train,y_train))
        test_score.append(model.score(x_test,y_test))
        RMSE_train.append(np.sqrt(mean_squared_log_error(y_train,model.predict(x_train))))
        RMSE_test.append(RMSE)
    
    d = pd.DataFrame()
    d['train_score'] = train_score
    d['test_score'] = test_score
    d['RMSE_train'] = RMSE_train
    d['RMSE_test'] = RMSE_test
    print(d)
    return best_model

In [527]:
# lgb
def evl_lgb(df):
    folds = KFold(n_splits=10,shuffle=True)
    df = fe_train(df)
    train = df.drop('SalePrice', axis=1)
    target = df['SalePrice']

    train_score,test_score,RMSE_train,RMSE_test = [],[],[],[]
    best_model = 0
    best_RMSE = 100
    for train_idx,test_idx in folds.split(train,target):
        x_train,y_train = train.iloc[train_idx,:],target[train_idx]
        x_test,y_test = train.iloc[test_idx,:],target[test_idx]
        
        model = lgb().fit(x_train,y_train)
        
        RMSE = np.sqrt(mean_squared_log_error(y_test,model.predict(x_test)))
        
        if RMSE < best_RMSE:
            best_RMSE = RMSE
            best_model = model
        
        train_score.append(model.score(x_train,y_train))
        test_score.append(model.score(x_test,y_test))
        RMSE_train.append(np.sqrt(mean_squared_log_error(y_train,model.predict(x_train))))
        RMSE_test.append(RMSE)
    
    d = pd.DataFrame()
    d['train_score'] = train_score
    d['test_score'] = test_score
    d['RMSE_train'] = RMSE_train
    d['RMSE_test'] = RMSE_test
    print(d)
    return best_model

In [554]:
# to_excel
def to_excel():
    s = pd.DataFrame()
    s['Id'] = Test.Id
    s['SalePrice'] = model.predict(fe_train(Test.drop('Id',axis=1)))
    s.to_csv('submission.csv',index=False)

In [459]:
# merging
def merge():
    s = pd.DataFrame()
    s['Id'] = Test.Id
    cat_pre = cat_model.predict(fe_train(Test.drop('Id',axis=1)))
    xgb_pre = xgb_model.predict(fe_train(Test.drop('Id',axis=1)))
    lgb_pre = lgb_model.predict(fe_train(Test.drop('Id',axis=1)))
    s['SalePrice'] = cat_pre*0.8 + xgb_pre*0.1 + lgb_pre*0.1
    s.to_csv('submission.csv',index=False)

In [ ]:
cat_model = evl_catb(df)

xgb_model = evl_xgb(df)

lgb_model = evl_lgb(df)

In [ ]:
model = evl_catb(df)

In [574]:
to_excel()  

In [ ]:
model = evl_lgb(df)

In [ ]:
model = evl(df) 
to_excel()  
model.save_model('catboost')